In [3]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from tqdm import tqdm

# Configuración general
data_dir = "./Skin_Cancer_Dataset_86porc_ResNet101"
num_classes = 7
batch_size = 64
num_epochs = 60
learning_rate = 1e-4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformaciones (data augmentation para entrenamiento)
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Transformación para validación y test
eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Carga de datasets
train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=train_transform)
valid_dataset = datasets.ImageFolder(os.path.join(data_dir, "valid"), transform=eval_transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=eval_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Cargar modelo base
model = models.resnet101(weights="IMAGENET1K_V1")

# Reemplazar capa final
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Congelar todas las capas excepto la última
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

# Loss y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

best_acc = 0.0

print("\n📌 Fase 1: entrenamiento solo de la capa final")
for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/5"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = correct / total
    scheduler.step(acc)
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Valid Acc: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_model.pt")
        print("✅ Nuevo mejor modelo guardado")

# Descongelar todo el modelo para fine-tuning
print("\n🔓 Fase 2: fine-tuning de todo el modelo")

for param in model.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

for epoch in range(5, num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = correct / total
    scheduler.step(acc)
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Valid Acc: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "best_model.pt")
        print("✅ Nuevo mejor modelo guardado")

print(f"\n🏁 Entrenamiento terminado. Mejor accuracy en validación: {best_acc:.4f}")


📌 Fase 1: entrenamiento solo de la capa final


Epoch 1/5: 100%|██████████| 109/109 [00:15<00:00,  6.83it/s]


Epoch 1, Loss: 128.6367, Valid Acc: 0.6704
✅ Nuevo mejor modelo guardado


Epoch 2/5: 100%|██████████| 109/109 [00:16<00:00,  6.80it/s]


Epoch 2, Loss: 109.8821, Valid Acc: 0.6876
✅ Nuevo mejor modelo guardado


...

Epoch 58, Loss: 26.7600, Valid Acc: 0.8519


Epoch 59/60: 100%|██████████| 109/109 [00:26<00:00,  4.19it/s]


Epoch 59, Loss: 26.9971, Valid Acc: 0.8504


Epoch 60/60: 100%|██████████| 109/109 [00:26<00:00,  4.18it/s]


Epoch 60, Loss: 26.8727, Valid Acc: 0.8488

🏁 Entrenamiento terminado. Mejor accuracy en validación: 0.8539
